In [1]:
# Import required libraries
import requests
import pandas as pd
import numpy as np
import re
import json
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import spacy
from spacy import displacy
from collections import defaultdict

# Load NLP models
nlp = spacy.load("en_core_web_sm")
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Entity types to classify
entity_types = ["corporation", "non-profit", "shell company", "government agency", "financial intermediary", "PEP"]

# Sample risk factors (would be expanded in real implementation)
risk_factors = {
    "sanctioned": 0.9,
    "shell_company": 0.8,
    "pep_associated": 0.7,
    "tax_haven": 0.6,
    "high_risk_country": 0.5,
    "anonymous_ownership": 0.7
}

# Helper functions for API integrations
def search_opencorporates(entity_name):
    """Search OpenCorporates API for company information"""
    try:
        url = f"https://api.opencorporates.com/v0.4/companies/search?q={entity_name}"
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        return None
    except:
        return None

def check_sanctions(entity_name):
    """Check if entity is on sanctions lists (mock implementation)"""
    # In real implementation, would use OFAC API or similar
    sanctioned_entities = ["oceanic holdings llc", "viktor petrov", "ali al-mansoori"]
    return entity_name.lower() in sanctioned_entities

def check_pep(entity_name):
    """Check if entity is a PEP (mock implementation)"""
    # In real implementation, would use World Bank PEP list
    peps = ["ali al-mansoori", "maria gonzalez"]
    return entity_name.lower() in peps

def get_wikipedia_info(entity_name):
    """Get Wikipedia summary for entity (mock implementation)"""
    # In real implementation, would use Wikipedia API
    return None

# Entity processing functions
def extract_entities(text):
    """Extract entities from text using spaCy"""
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        if ent.label_ in ["ORG", "PERSON"]:
            entities.append(ent.text)
    return list(set(entities))

def classify_entity(entity_name):
    """Classify entity type using zero-shot classification"""
    result = classifier(entity_name, entity_types)
    return result['labels'][0]

def calculate_risk_score(entity_name, entity_type):
    """Calculate risk score based on entity attributes"""
    score = 0.3  # Base score

    # Check sanctions
    if check_sanctions(entity_name):
        score = max(score, risk_factors["sanctioned"])

    # Check PEP status
    if check_pep(entity_name):
        score = max(score, risk_factors["pep_associated"])

    # Additional risk factors based on entity type
    if entity_type == "shell company":
        score = max(score, risk_factors["shell_company"])

    # Check for tax haven countries (simplified)
    tax_havens = ["cayman", "panama", "bvi", "british virgin islands", "switzerland"]
    if any(haven in entity_name.lower() for haven in tax_havens):
        score = max(score, risk_factors["tax_haven"])

    return round(score, 2)

def gather_evidence(entity_name):
    """Gather supporting evidence for entity"""
    evidence = []

    # Check OpenCorporates
    oc_data = search_opencorporates(entity_name)
    if oc_data and oc_data.get('results', {}).get('companies'):
        evidence.append("OpenCorporates")

    # Check sanctions
    if check_sanctions(entity_name):
        evidence.append("Sanctions List")

    # Check PEP
    if check_pep(entity_name):
        evidence.append("PEP List")

    # Check Wikipedia
    wiki_data = get_wikipedia_info(entity_name)
    if wiki_data:
        evidence.append("Wikipedia")

    if not evidence:
        evidence.append("No public records found")

    return evidence

def process_transaction(transaction):
    """Process a single transaction record"""
    if isinstance(transaction, dict):
        # Process unstructured data
        text_parts = [
            transaction.get('Sender', {}).get('Name', ''),
            transaction.get('Receiver', {}).get('Name', ''),
            transaction.get('Additional Notes', '')
        ]
        text = ' '.join(text_parts)
        transaction_id = transaction.get('Transaction ID', 'UNKNOWN')
    else:
        # Process structured data (Pandas Series)
        text_parts = [
            transaction['Payer Name'],
            transaction['Receiver Name'],
            transaction['Transaction Details']
        ]
        text = ' '.join([str(x) for x in text_parts])
        transaction_id = transaction['Transaction ID']

    entities = extract_entities(text)
    results = []

    for entity in entities:
        entity_type = classify_entity(entity)
        risk_score = calculate_risk_score(entity, entity_type)
        evidence = gather_evidence(entity)
        confidence = min(0.95, risk_score + 0.3)  # Simple confidence heuristic

        # Generate reason text
        reason_parts = []
        if check_sanctions(entity):
            reason_parts.append(f"{entity} is on sanctions list")
        if check_pep(entity):
            reason_parts.append(f"{entity} is a Politically Exposed Person")
        if entity_type == "shell company":
            reason_parts.append(f"{entity} appears to be a shell company")

        reason = ". ".join(reason_parts) if reason_parts else f"No high-risk indicators found for {entity}"

        results.append({
            "Transaction ID": transaction_id,
            "Extracted Entity": entity,
            "Entity Type": entity_type,
            "Risk Score": risk_score,
            "Supporting Evidence": evidence,
            "Confidence Score": round(confidence, 2),
            "Reason": reason
        })

    return results

# Main processing function
def analyze_transactions(data):
    """Analyze all transactions in the dataset"""
    all_results = []

    if isinstance(data, pd.DataFrame):
        # Process structured data
        for _, row in data.iterrows():
            all_results.extend(process_transaction(row))
    elif isinstance(data, list):
        # Process unstructured data
        for transaction in data:
            all_results.extend(process_transaction(transaction))

    return all_results

# Sample data processing
if __name__ == "__main__":
    # Create sample structured data
    structured_data = pd.DataFrame([
        ["TXN001", "Acme Corp", "SovCo Capital Partners", "Payment for services rendered", 500000, "USA"],
        ["TXN002", "Global Health Foundation", "Save the Children", "Grant disbursement", 2000000, "UK"],
        ["TXN003", "XYZ Ltd", "ABC GmbH", "Purchase of office supplies", 15000, "Germany"],
        ["TXN004", "Green Earth Org", "CCMI", "Environmental project funding", 750000, "Cayman Islands"],
        ["TXN005", "Oceanic Holdings LLC", "Alas Chiricanas", "Offshore investment", 5000000, "Panama"]
    ], columns=["Transaction ID", "Payer Name", "Receiver Name", "Transaction Details", "Amount", "Receiver Country"])

    # Create sample unstructured data
    unstructured_data = [
        {
            "Transaction ID": "TXN-2023-5A9B",
            "Sender": {
                "Name": "Global Horizons Consulting LLC",
                "Account": "IBAN CH56 0483 5012 3456 7800 9",
                "Address": "Rue du Marché 17, Geneva, Switzerland",
                "Notes": "Consulting fees for project Aurora"
            },
            "Receiver": {
                "Name": "Bright Future Nonprofit Inc",
                "Account": "987654321",
                "Address": "P.O. Box 1234, George Town, Cayman Islands",
                "Tax ID": "KY-4567B"
            },
            "Amount": 49850.00,
            "Additional Notes": "Urgent transfer approved by Mr. Ali Al-Mansoori (Director). Linked invoice missing. Processed via intermediary Quantum Holdings Ltd (BVI)."
        },
        {
            "Transaction ID": "TXN-2023-7C2D",
            "Sender": {
                "Name": "Quantum Holdings Ltd",
                "Account": "VGB2BVIR024987654321",
                "Beneficiary Owner": "Maria Gonzalez"
            },
            "Receiver": {
                "Name": "Golden Sands Trading FZE",
                "Account": "AE45033000012345678901",
                "Registration": "UAE Free Zone License #789-FZ"
            },
            "Amount": 950000.00,
            "Additional Notes": "Approver: Mr. Viktor Petrov (Linked to OFAC SDN List entry #9876, 2022)."
        }
    ]

    # Analyze both datasets
    structured_results = analyze_transactions(structured_data)
    unstructured_results = analyze_transactions(unstructured_data)

    # Combine and print results
    all_results = structured_results + unstructured_results
    print(json.dumps(all_results, indent=2))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[
  {
    "Transaction ID": "TXN001",
    "Extracted Entity": "Acme Corp SovCo Capital Partners Payment",
    "Entity Type": "corporation",
    "Risk Score": 0.3,
    "Supporting Evidence": [
      "No public records found"
    ],
    "Confidence Score": 0.6,
    "Reason": "No high-risk indicators found for Acme Corp SovCo Capital Partners Payment"
  },
  {
    "Transaction ID": "TXN002",
    "Extracted Entity": "Global Health Foundation Save",
    "Entity Type": "non-profit",
    "Risk Score": 0.3,
    "Supporting Evidence": [
      "No public records found"
    ],
    "Confidence Score": 0.6,
    "Reason": "No high-risk indicators found for Global Health Foundation Save"
  },
  {
    "Transaction ID": "TXN003",
    "Extracted Entity": "XYZ Ltd ABC GmbH Purchase",
    "Entity Type": "corporation",
    "Risk Score": 0.3,
    "Supporting Evidence": [
      "No public records found"
    ],
    "Confidence Score": 0.6,
    "Reason": "No high-risk indicators found for XYZ Ltd ABC GmbH Purc